<a href="https://colab.research.google.com/github/ravichas/AMPL-Tutorial/blob/master/03_CHEMBL26_SCN5A_IC50_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Building a Graph Convolutional Network Model for Drug Response Prediction</h1>

The ATOM Modeling PipeLine (AMPL; https://github.com/ATOMconsortium/AMPL) is an open-source, modular, extensible software pipeline for building and sharing models to advance in silico drug discovery.

**Warning: This is an experimental notebook**

AMPL is using Python 3.6.7 while Colab is currently using 3.6.9 listed below so they do not currently match. Your mileage may vary.

## Please change your runtime to GPU 

Go to **Runtime** --> Change **runtime type** to "GPU"


## Goal: Use AMPL for predicting binding affinities -IC50 values- of ligands that could bind to human **Sodium channel protein type 5 subunit alpha** protein using Graph Convolutional Network Model. chEMBL database is the source of the binding affinities (pIC50).

In this notebook, we describe the following steps using AMPL:

1.   Read a ML ready dataset
2.   Fit a Graph Convolutional model
3.   Predict pIC50 values of withheld compounds

## Set up
We first import the AMPL modules for use in this notebook.

The relevant AMPL modules for this example are listed below:

|module|Description|
|-|-|
|`atomsci.ddm.pipeline.model_pipeline`|The model pipeline module is used to fit models and load models for prediction.|
|`atomsci.ddm.pipeline.parameter_parser`|The parameter parser reads through pipeline options for the model pipeline.|
|`atomsci.ddm.utils.curate_data`|The curate data module is used for data loading and pre-processing.|
|`atomsci.ddm.utils.struct_utils`|The structure utilities module is used to process loaded structures.|


import atomsci.ddm.pipeline.compare_models as cmp
import atomsci.ddm.pipeline.model_pipeline as mp
import atomsci.ddm.pipeline.parameter_parser as parse

## Install AMPL 

In [1]:
import requests

# Copy AMPL install script
url='https://raw.githubusercontent.com/ravichas/AMPL-Tutorial/master/config/install_AMPL_GPU.sh'

downloaded_obj = requests.get(url)
with open("install_AMPL_GPU.sh", "wb") as file:
    file.write(downloaded_obj.content)


# Install AMPL
!chmod u+x install_AMPL_GPU.sh
!time ./install_AMPL_GPU.sh
import sys
if '/content/AMPL/lib/python3.6/site-packages' not in sys.path:
    sys.path.insert(1, '/content/AMPL/lib/python3.6/site-packages')

--2020-10-13 20:04:57--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93052469 (89M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  88.74M   220MB/s    in 0.4s    

2020-10-13 20:04:58 (220 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [93052469/93052469]

PREFIX=/content/AMPL
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /content/AMPL

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - ca-certificates==2020.1.1=0
    - certifi==2020.4.5.1=py38_0
    - cffi==1.14.0=py38he30daa8_1
    - chardet==3.0.4=py38_1003
    - conda-package-handling==1.6.1=py38h7b6447c_0
    - conda==4.8.3=py38_0
 

In [3]:
!find . -name transformations.py 

./AMPL/lib/python3.6/site-packages/atomsci/ddm/pipeline/transformations.py
./github/AMPL.build/ampl/lib/atomsci/ddm/pipeline/transformations.py
./github/AMPL/atomsci/ddm/pipeline/transformations.py


In [4]:
# There is a problem with the UMAP package, so temporarily remove umap import
%%bash
cat << EOF > ampl.patch
--- transformations.py  2020-09-14 17:08:22.225747322 -0700
+++ transformations_patched.py  2020-09-14 17:08:07.869651225 -0700
@@ -9,7 +9,7 @@

 import numpy as np
 import pandas as pd
-import umap
+# import umap

 import deepchem as dc
 from deepchem.trans.transformers import Transformer, NormalizationTransformer
EOF

patch -N /AMPL/lib/python3.6/site-packages/atomsci/ddm/pipeline/transformations.py

**The following code-chunk will take about 6-7 minutes**

**Note: Can start here if the Colab runtime is restarted. Environment is saved in the Colab runtime file system.**

In [5]:
# Load AMPL in this notebook
import sys

site_packages_path = '/content/AMPL/lib/python3.6/site-packages'
if site_packages_path not in sys.path:
  sys.path.insert(1, site_packages_path)
sys.path

['',
 '/content/AMPL/lib/python3.6/site-packages',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [6]:
# There is a problem with the previously imported cffi, so delete it and load it with AMPL instead
if 'cffi' in sys.modules:
  del sys.modules['cffi']

In [7]:
import atomsci.ddm.pipeline.compare_models as cmp
import atomsci.ddm.pipeline.model_pipeline as mp
import atomsci.ddm.pipeline.parameter_parser as parse

/content/AMPL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/content/AMPL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/content/AMPL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/content/AMPL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (

In [8]:
import os
os.mkdir('chembl_activity_models')

**Upload data file ChEMBL26_SCN5A_IC50_human_ml_ready.csv to Colab now.**

In [9]:
import pandas as pd
import requests
import io
url = 'https://raw.githubusercontent.com/ravichas/AMPL-Tutorial/master/datasets/ChEMBL26_SCN5A_IC50_human_ml_ready.csv'
download = requests.get(url).content
df = pd.read_csv(url, index_col=0)
# Reading the downloaded content and turning it into a pandas dataframe
df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df.iloc[0:5, 0:5]
df.to_csv('ChEMBL26_SCN5A_IC50_human_ml_ready.csv', index=False)

In [10]:
split_config = {
"script_dir": "/content/AMPL/atomsci/ddm",
"dataset_key" : "/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv",
"datastore": "False",
"split_only": "True",
"splitter": "scaffold",
"split_valid_frac": "0.15",
"split_test_frac": "0.15",
"previously_split": "False",
"prediction_type": "regression",
"response_cols" : "pIC50",
"id_col": "compound_id",
"smiles_col" : "base_rdkit_smiles",
"result_dir": "/content/chembl_activity_models",
"system": "LC",
"transformers": "True",
"model_type": "NN",
"featurizer": "graphconv",
"descriptor_type": "graphconv",
"learning_rate": "0.0007",
"layer_sizes": "64,64,32",
"dropouts" : "0.0,0.0,0.0",
"save_results": "False",
"max_epochs": "100",
"verbose": "True"
}

In [11]:
split_params = parse.wrapper(split_config)

In [12]:
split_model = mp.ModelPipeline(split_params)

In [13]:
split_uuid = split_model.split_dataset()
split_uuid

number of features: 75


2020-10-13 20:29:00,921 Splitting data by scaffold
2020-10-13 20:29:20,306 Dataset split table saved to /content/ChEMBL26_SCN5A_IC50_human_ml_ready_train_valid_test_scaffold_480a8986-e3f7-4dae-9169-848ada884e81.csv


'480a8986-e3f7-4dae-9169-848ada884e81'

In [14]:
!pip install --upgrade gspread

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


## Train the mode (~ 10 minutes)

In [15]:
train_config = {
"script_dir": "/content/AMPL/atomsci/ddm",
"dataset_key" : "/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv",
"datastore": "False",
"uncertainty": "False",
"splitter": "scaffold",
"split_valid_frac": "0.15",
"split_test_frac": "0.15",
"previously_split": "True",
"split_uuid": "{}".format(split_uuid),
"prediction_type": "regression",
"response_cols" : "pIC50",
"id_col": "compound_id",
"smiles_col" : "base_rdkit_smiles",
"result_dir": "/content/chembl_activity_models",
"system": "LC",
"transformers": "True",
"model_type": "NN",
"featurizer": "graphconv",
"descriptor_type": "graphconv",
"learning_rate": "0.0007",
"layer_sizes": "64,64,32",
"dropouts" : "0.0,0.0,0.0",
"save_results": "False",
"max_epochs": "100",
"verbose": "True"
}

In [19]:
train_params = parse.wrapper(train_config)

In [20]:
train_model = mp.ModelPipeline(train_params)

## Train_model took ~ 18 minutes on a GPU  (~ 30 minutes on a CPU)

In [22]:
train_model.train_model()

number of features: 75


2020-10-13 20:31:31,619 Previous dataset split restored


n_cnt [1729.]
y_means [5.15460889]
y_stds [0.64359373]
TIMING: dataset construction took 1.246 s
Loading dataset from disk.
TIMING: dataset construction took 0.213 s
Loading dataset from disk.
TIMING: dataset construction took 0.301 s
Loading dataset from disk.


/content/AMPL/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/bce8232a-7823-40ab-86db-a8299c6f3408/model/model-25


...

INFO:tensorflow:Restoring parameters from /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/bce8232a-7823-40ab-86db-a8299c6f3408/model/model-2450


2020-10-13 20:47:53,287 Restoring parameters from /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/bce8232a-7823-40ab-86db-a8299c6f3408/model/model-2450


INFO:tensorflow:Restoring parameters from /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/bce8232a-7823-40ab-86db-a8299c6f3408/model/model-2475


2020-10-13 20:48:01,007 Restoring parameters from /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/bce8232a-7823-40ab-86db-a8299c6f3408/model/model-2475
/content/AMPL/lib/python3.6/site-packages/atomsci/ddm/pipeline/model_wrapper.py:767: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  chkpt_dict = yaml.load(chkpt_in.read())
2020-10-13 20:49:31,976 Wrote model metadata to file /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/bce8232a-7823-40ab-86db-a8299c6f3408/model_metadata.json
2020-10-13 20:49:32,207 Wrote model metrics to file /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/bce8232a-7823-40ab-86db-a8299c6f3408/model_metrics.json


Wrote model tarball to /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready_model_bce8232a-7823-40ab-86db-a8299c6f3408.tar.gz


In [23]:
perf_df = cmp.get_filesystem_perf_results('/content/chembl_activity_models', pred_type='regression')
perf_df

Found data for 1 models under /content/chembl_activity_models


,model_uuid,model_type,dataset_key,featurizer,splitter,model_score_type,feature_transform_type,learning_rate,dropouts,layer_sizes,best_epoch,max_epochs,rf_estimators,rf_max_features,rf_max_depth,model_choice_score,train_r2_score,train_rms_score,train_mae_score,valid_r2_score,valid_rms_score,valid_mae_score,test_r2_score,test_rms_score,test_mae_score
0,bce8232a-7823-40ab-86db-a8299c6f3408,NN,/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv,graphconv,scaffold,r2,normalization,0.0007,"0.00,0.00,0.00","64,64,32",30,100,NaN,NaN,NaN,0.346238,0.755697,0.316926,0.233108,0.346238,0.475164,0.356624,0.28816,0.592226,0.390882


In [ ]:
11:26